# Mappointer
#### Wes Mayne & Billy Tang
*Using Pandas,SLQAlchemy and Google Maps API, pull site geocode data for established TMS sites and write the retrieved mappoint back to the database.*


In [2]:
#Modules
import requests
import pandas as pd
import sqlalchemy

In [3]:
#SQL Alchemy
user = ''
password = ''
servername = ''
database = ''

column = 'CITYTOWN'
condition = 'TAS'

connection_string = f'mssql+pymssql://{user}:{password}@{servername}/{database}'
engine = sqlalchemy.create_engine(connection_string)
conn = engine.connect()

sql = f"select SITEID, SITENAME, MPID, ADDRESS1, SUBURB, CITYTOWN, POSTALCODE from SITE where {column} = '{condition}' AND STARTDATE >= CAST(GETDATE() AS DATE)"
#sql = f"select top 3 SITEID, SITENAME, MPID, ADDRESS1, SUBURB, CITYTOWN, POSTALCODE from SITE where {column} = '{condition}'"

In [4]:
#API Info, Add your own
API_KEY = ''

In [5]:
#FUNction
def get_mappoint(address):
    URL = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_KEY}'
    r = requests.get(URL)
    data = r.json()
    loc_type = data['results'][0]['geometry']['location_type']
    lat = data['results'][0]['geometry']['location']['lat']
    lng = data['results'][0]['geometry']['location']['lng']
    
    for address in site['search_string']:
        return lat, lng, loc_type

In [6]:
#Pandas 

site = pd.read_sql(sql, conn)

#Concat columns and add '+' in between
site['search_string'] = site['ADDRESS1'] + ', ' + site['SUBURB'] + ', ' + site['CITYTOWN']
site['search_string'] = site['search_string'].apply(lambda x: str(x).replace(' ', '+'))



#Apply get_mappoint function to entire search_string series
site['results'] = site['search_string'].apply(get_mappoint)

#split 'results' series into 3 new columns and rename headers
results = site['results'].apply(pd.Series)
results = results.rename(columns = lambda x : 'results' + str(x))

#rename to proper names
results = results.rename(columns = {"results0": "Y", 
                                  "results1":"X", 
                                  "results2": "loc_type"}) 

#join columns back into original dataframe
site = pd.concat([site[:], results[:]], axis=1)

#select relevant columns from site
clean_site = site[['SITENAME', 'SITEID','MPID', 'X', 'Y', 'loc_type']]

In [7]:
#add column for SQL mappoint INSERT query
clean_site['mappoint_query'] = 'INSERT INTO MAPPOINT(MAPPOINT, X, Y, MPCLASSID, APPROXIMATE, SNAP_X, SNAP_Y) VALUES('+"'"+clean_site['SITENAME']+"','"+clean_site['X'].map(str)+"','"+clean_site['Y'].map(str)+"','-1', 'F','"+clean_site['X'].map(str)+"','"+clean_site['Y'].map(str)+"')"     

c:\users\wmayne\envs\env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
#Export to Excel file 
export = clean_site.to_excel(r'C:\Users\wmayne\OneDrive - Lion Pty Ltd\Documents\site.xlsx', index = None, header=True)